### WORK TO DONE:
1. Identify problems
2. Define Success
3. Collect and prepare the data
4. Analyze and profile the data
5. Communicate findings
6. Advocate for action

---
 Report Category: The high-level grouping for each report.
 Geographic Subdivision: The individual schools or geographic subregion for each of the main categories.
 Grade: The school grade for which the test was administered.

1.   Report Category: The high-level grouping for each report.
2.   Geographic Subdivision: The individual schools or geographic subregion for each of the main categories
3. Grade: The school grade for which the test was administered.
4. Year: The year the test was administered.
5.  Number Tested: Total number of students tested.
6. Mean Scale Score: Average score of total students tested.
7. Num Level 1: Number of students who scored in Level 1 range.
8. Pct Level 1: Percentage of students who scored in Level 1 range.
9. Num Level 2: Number of students who scored in Level 2 range.
10. Pct Level 2: Percentage of students who scored in Level 2 range.
11. Num Level 3: Number of students who scored in Level 3 range.
12. Pct Level 3: Percentage of students who scored in Level 3 range.
13. Num Level 4: Number of students who scored in Level 4 range.
14. Pct Level 4: Percentage of students who scored in Level 4 range.
15. Num Level 3 and 4: Number of students who scored in Level 3 and 4 range combined.
16. Pct Level 3 and 4: Percentage of students who scored in Level 3 and 4 range



---



In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import scipy.stats as stats
import random
import matplotlib.pyplot as plt
from scipy.stats import norm

# 1. Identify problems
+ What specific problem am i trying to solve?
+ Who are the key stakeholders, and how will this help them?
+ Does this allign with broader business priorities and strategy?

In [3]:
company_list = pd.read_csv("D:\Project DA\Data\Company List.csv",delimiter=';')
company_list.head(5)

,Name,Symbol,Holdings,Market Price,Value,Stake
0,"Amazon.com, Inc.",AMZN,"533,3","$3,272.71","$1,745,336,243",0.1%
1,American Express Company,AXP,"151,610,700",$104.91,"$15,905,478,537",18.8%
2,Apple Inc.,AAPL,"1,003,466,264",$119.02,"$119,432,554,741",5.9%
3,Axalta Coating Systems Ltd,AXTA,"23,535,041",$25.64,"$603,438,451",10.0%
4,Bank of America Corp,BAC,"1,032,852,006",$24.24,"$25,036,332,625",11.9%


In [4]:
company_list['Market Price'] = company_list['Market Price'].str.replace('$', '')
company_list['Value'] = company_list['Value'].str.replace('$','')
company_list['Stake'] = company_list['Stake'].str.replace('%','')
company_list.head(5)

,Name,Symbol,Holdings,Market Price,Value,Stake
0,"Amazon.com, Inc.",AMZN,"533,3","3,272.71","1,745,336,243",0.1
1,American Express Company,AXP,"151,610,700",104.91,"15,905,478,537",18.8
2,Apple Inc.,AAPL,"1,003,466,264",119.02,"119,432,554,741",5.9
3,Axalta Coating Systems Ltd,AXTA,"23,535,041",25.64,"603,438,451",10.0
4,Bank of America Corp,BAC,"1,032,852,006",24.24,"25,036,332,625",11.9


In [5]:
stock_profile = pd.read_csv('D:\Project DA\Data\stock_profile.csv',delimiter=';')
stock_profile.head(5)

,Name,Symbol,Sector,Industry,Num_Employees
0,Amazon.com Inc.,AMZN,Consumer Cyclical,Internet Retail,1125300.0
1,American Express Company,AXP,Financial Services,Credit Services,64000.0
2,Apple Inc.,AAPL,Technology,Consumer Electronics,147000.0
3,Axalta Coating Systems Ltd.,AXTA,Basic Materials,Specialty Chemicals,13000.0
4,Bank of America Corporation,BAC,Financial Services,Banks—Diversified,211000.0


In [6]:
df = pd.concat([company_list,stock_profile],axis=1)
df = df.loc[:, ~df.columns.duplicated()]
print(df.head(5))

                         Name Symbol       Holdings Market Price  \
0            Amazon.com, Inc.   AMZN          533,3     3,272.71   
1    American Express Company    AXP    151,610,700       104.91   
2                  Apple Inc.   AAPL  1,003,466,264       119.02   
3  Axalta Coating Systems Ltd   AXTA     23,535,041        25.64   
4        Bank of America Corp    BAC  1,032,852,006        24.24   

             Value Stake              Sector              Industry  \
0    1,745,336,243   0.1   Consumer Cyclical       Internet Retail   
1   15,905,478,537  18.8  Financial Services       Credit Services   
2  119,432,554,741   5.9          Technology  Consumer Electronics   
3      603,438,451  10.0     Basic Materials   Specialty Chemicals   
4   25,036,332,625  11.9  Financial Services     Banks—Diversified   

   Num_Employees  
0      1125300.0  
1        64000.0  
2       147000.0  
3        13000.0  
4       211000.0  


#DEALING WITH MISSING VALUE

In [7]:
# Define total number of missing values
df.isnull().sum()

Name             0
Symbol           0
Holdings         0
Market Price     0
Value            0
Stake            1
Sector           3
Industry         3
Num_Employees    8
dtype: int64

In [8]:
df.dropna(inplace=True)  # Drops rows with any missing values

# Analysis Data

In [161]:
# Tìm top 3 công ti được đầu tư nhiều nhất:
# Convert the 'Value' column to a string type.
df['Value'] = df['Value'].astype(str)
# Replace commas with empty strings.
df['Value'] = df['Value'].str.replace(',', '')
# Convert the 'Value' column to a float type.
df['Value'] = df['Value'].astype(float)
# Sort the DataFrame by the 'Value' column.
df_sorted = df.sort_values(by='Value', ascending=False)
# Print the top 3 rows of the sorted DataFrame.
print(df_sorted.head(3))

                   Name Symbol       Holdings Market Price         Value  \
2            Apple Inc.   AAPL  1,003,466,264       119.02  1.194326e+11   
4  Bank of America Corp    BAC  1,032,852,006        24.24  2.503633e+10   
9          Coca-Cola Co     KO    400,000,000        50.03  2.001200e+10   

  Stake              Sector                 Industry  Num_Employees  
2   5.9          Technology     Consumer Electronics       147000.0  
4  11.9  Financial Services        Banks—Diversified       211000.0  
9   9.3  Consumer Defensive  Beverages—Non-Alcoholic        86200.0  


In [129]:
AAPL = pd.read_csv('/content/AAPL.csv')
AAPL.head(5)

,Date,High,Low,Open,Close,Volume,Adj Close
0,2007-01-03,3.092143,2.925000,3.081786,2.992857,1.238320e+09,2.560825
1,2007-01-04,3.069643,2.993572,3.001786,3.059286,8.472604e+08,2.617664
2,2007-01-05,3.078571,3.014286,3.063214,3.037500,8.347416e+08,2.599023
3,2007-01-08,3.090357,3.045714,3.070000,3.052500,7.971068e+08,2.611857
4,2007-01-09,3.320714,3.041071,3.087500,3.306072,3.349298e+09,2.828825
